In [1]:
import pandas as pd
import numpy as np

import requests

import os
import time
from datetime import datetime
import sys
import yaml
import gc


from data_loader import *
from utils import *
from nn import *


import seaborn as sns
sns.set_style("darkgrid")
plt.rcParams.update({
    'axes.facecolor': '#1e1e1e',
    'figure.facecolor': '#1e1e1e',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'white',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'axes.grid': True,
    'grid.color': 'gray'
})

pd.set_option("display.max_column",None)
print(os.getcwd())


%load_ext autoreload
%reload_ext autoreload
%autoreload 2


def auto_reload():
    %load_ext autoreload
    %reload_ext autoreload
    %autoreload 2

/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app


In [2]:
with open("config.yaml",'r') as f:
    config = yaml.safe_load(f)

home_dir = config['HOME_DIRECTORY']
home_dir

'/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG'

In [3]:
# Model File Configs:
# Every Run of the notebook is logged in to a submodel folder for it

############################################################################
# Please set output path to the project directory where it is uncompressed #
############################################################################

project_path = home_dir + "/outputs/models/"




output_path = ""

submodel_name = ""

encoding_path = ""
feature_report_path = ""
def create_submodel(model_name:str):
    author = "EJ"
    global submodel_name
    submodel_name = datetime.now().strftime("%d_%H_%M") + "_"+model_name
    global output_path
    output_path = project_path+submodel_name
    global encoding_path
    encoding_path = output_path+"/encodings/"
    global feature_report_path
    feature_report_path = output_path+"/feature_report/"
    os.mkdir(output_path)
    os.mkdir(encoding_path)
    os.mkdir(feature_report_path)


In [4]:
create_submodel("test")

# API Football:
https://www.api-football.com

![PYTHON LOGO](https://www.api-football.com/public/img/news/archi-beta.jpg)

In [5]:
leagues_dat = get_leagues(home_dir +"/data/Leagues/leagues.parquet")
leagues_dat[['league_id','league_name','country_name']]

,league_id,league_name,country_name
0,4,Euro Championship,World
1,21,Confederations Cup,World
2,61,Ligue 1,France
3,144,Jupiler Pro League,Belgium
4,71,Serie A,Brazil
...,...,...,...
1128,1131,Super Cup,Mongolia
1129,1132,AFC Challenge League,World
1130,1133,Goiano - 3,Brazil
1131,1001,CONCACAF Women U20,World


# Leagues subset:

In [6]:
# Configs
major_leagues = ["Premier League","La Liga","Serie A","Bundesliga","Eredivisie","Ligue 1"]
major_countries = ["England","Spain","Italy","Germany","Netherlands","France","Brazil"]
teams = ["Liverpool","Wolves"] # teams to pull players data of
seasons = [2022,2021,2023,2024] # seasons to pull players and teams stats of



leagues_subset = leagues_dat[leagues_dat.league_name.isin(major_leagues) & leagues_dat.country_name.isin(major_countries)] # league ID to pull from, current values: {39:premier league}, Add to dictionary as needed

In [7]:
leagues_subset

,seasons,league_id,league_name,league_type,league_logo,country_name,country_code,country_flag
2,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg
4,"[{'coverage': {'fixtures': {'events': True, 'l...",71,Serie A,League,https://media.api-sports.io/football/leagues/7...,Brazil,BR,https://media.api-sports.io/flags/br.svg
5,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg
6,"[{'coverage': {'fixtures': {'events': True, 'l...",78,Bundesliga,League,https://media.api-sports.io/football/leagues/7...,Germany,DE,https://media.api-sports.io/flags/de.svg
7,"[{'coverage': {'fixtures': {'events': True, 'l...",135,Serie A,League,https://media.api-sports.io/football/leagues/1...,Italy,IT,https://media.api-sports.io/flags/it.svg
8,"[{'coverage': {'fixtures': {'events': True, 'l...",88,Eredivisie,League,https://media.api-sports.io/football/leagues/8...,Netherlands,NL,https://media.api-sports.io/flags/nl.svg
10,"[{'coverage': {'fixtures': {'events': True, 'l...",140,La Liga,League,https://media.api-sports.io/football/leagues/1...,Spain,ES,https://media.api-sports.io/flags/es.svg


# Read All fixtures data

In [60]:
teams_dat = pd.read_parquet(home_dir + "/data/Teams/team_league.parquet")

In [55]:
all_fixtures = get_team_fixtures("Liverpool",2024)

Pulling for Liverpool,2024 with league_id: 39


In [56]:
all_fixtures.head()

,fixture_id,fixture_referee,fixture_timezone,fixture_date,fixture_timestamp,fixture_periods_first,fixture_periods_second,fixture_venue_id,fixture_venue_name,fixture_venue_city,fixture_status_long,fixture_status_short,fixture_status_elapsed,fixture_status_extra,league_id,league_name,league_country,league_logo,league_flag,league_season,league_round,league_standings,teams_home_id,teams_home_name,teams_home_logo,teams_home_winner,teams_away_id,teams_away_name,teams_away_logo,teams_away_winner,goals_home,goals_away,score_halftime_home,score_halftime_away,score_fulltime_home,score_fulltime_away,score_extratime_home,score_extratime_away,score_penalty_home,score_penalty_away
1,1208022,T. Robinson,UTC,2024-08-17T11:30:00+00:00,1723894200,1723894200,1723897800,545,Portman Road,"Ipswich, Suffolk",Match Finished,FT,90,NaN,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg,2024,Regular Season - 1,True,57,Ipswich,https://media.api-sports.io/football/teams/57.png,False,40,Liverpool,https://media.api-sports.io/football/teams/40.png,True,0,2,0,0,0,2,None,None,None,None
19,1208036,S. Attwell,UTC,2024-08-25T15:30:00+00:00,1724599800,1724599800,1724603400,550,Anfield,Liverpool,Match Finished,FT,90,NaN,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg,2024,Regular Season - 2,True,40,Liverpool,https://media.api-sports.io/football/teams/40.png,True,55,Brentford,https://media.api-sports.io/football/teams/55.png,False,2,0,1,0,2,0,None,None,None,None
29,1208047,A. Taylor,UTC,2024-09-01T15:00:00+00:00,1725202800,1725202800,1725206400,556,Old Trafford,Manchester,Match Finished,FT,90,NaN,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg,2024,Regular Season - 3,True,33,Manchester United,https://media.api-sports.io/football/teams/33.png,False,40,Liverpool,https://media.api-sports.io/football/teams/40.png,True,0,3,0,2,0,3,None,None,None,None
32,1208056,M. Oliver,UTC,2024-09-14T14:00:00+00:00,1726322400,1726322400,1726326000,550,Anfield,Liverpool,Match Finished,FT,90,NaN,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg,2024,Regular Season - 4,True,40,Liverpool,https://media.api-sports.io/football/teams/40.png,False,65,Nottingham Forest,https://media.api-sports.io/football/teams/65.png,True,0,1,0,0,0,1,None,None,None,None
42,1208066,T. Harrington,UTC,2024-09-21T14:00:00+00:00,1726927200,1726927200,1726930800,550,Anfield,Liverpool,Match Finished,FT,90,NaN,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg,2024,Regular Season - 5,True,40,Liverpool,https://media.api-sports.io/football/teams/40.png,True,35,Bournemouth,https://media.api-sports.io/football/teams/35.png,False,3,0,3,0,3,0,None,None,None,None


In [8]:
fixtures_dir = home_dir + "/data/Fixtures"

complete_data = pd.DataFrame()
for file in os.listdir(fixtures_dir):    
    dat = pd.read_parquet(os.path.join(fixtures_dir,file))
    complete_data = pd.concat([complete_data,dat],axis = 0)

complete_data = complete_data.reset_index()
complete_data.drop(columns = ['index'],inplace=True)

/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_50540/3256445224.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complete_data = pd.concat([complete_data,dat],axis = 0)
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_50540/3256445224.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complete_data = pd.concat([complete_data,dat],axis = 0)
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_50540/3256445224.py:6: FutureWarning: The behavior of DataFrame concatena

In [58]:
# Add Missed features from fixtures
complete_data = pd.merge(complete_data,all_fixtures[['fixture_id','fixture_date']],on= 'fixture_id',how = 'left')

In [59]:
complete_data.head(2)

,player_id,player_name,offsides,games_minutes,games_number,games_position,games_rating,games_captain,games_substitute,shots_total,shots_on,goals_total,goals_conceded,goals_assists,goals_saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved,fixture_id,team_goals_scored,team_non_penalty_goals_scored,team_goals_scored_half,team_goals_conceded,team_non_penalty_goals_conceded,team_goals_conceded_half,opponent,team_winner,team,outcome,dribble_success_rate,target_shot_conversion_perc,duels_won_perc,pass_accuracy_perc,outcome_num,win,fixture_date
0,47269,Álex Remiro,NaN,90.0,1,G,6.2,False,False,NaN,NaN,NaN,2,0.0,2.0,31.0,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,0.0,1208500,1,1.0,0,2,2.0,0,Rayo Vallecano,Rayo Vallecano,Real Sociedad,loss,NaN,NaN,NaN,83.870968,1,0,NaN
1,2202,Hamari Traoré,NaN,75.0,18,D,6.9,False,False,NaN,NaN,NaN,0,0.0,NaN,37.0,NaN,30.0,2.0,NaN,NaN,8.0,5.0,1.0,1.0,1.0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,1208500,1,1.0,0,2,2.0,0,Rayo Vallecano,Rayo Vallecano,Real Sociedad,loss,100.0,NaN,62.5,81.081081,1,0,NaN


In [9]:
# Data checks
complete_data['passes_accuracy'] = complete_data['passes_accuracy'].astype("float64")

# Targets
complete_data['outcome_num'] = pd.Categorical(complete_data.outcome).codes

complete_data['win'] = np.where(complete_data.outcome.str.lower() == 'win', 1,0)


In [10]:
# This is the dictionary that contains all information about the features    
dat_dict = find_data_types(complete_data,config['OUTCOME_COLS'] + ['outcome_num','outcome'])
dat_dict = pd.DataFrame(list(dat_dict.items()),columns =['feature','type'])

# differentiate modeling features
non_modeling_features = config['FIXTURE_COLS'] + config['OUTCOME_COLS'] + config['MISC_COLS'] + ['outcome_num']
dat_dict['modeling_feature'] = np.where(dat_dict['feature'].isin(non_modeling_features),0,1)
dat_dict['encoded'] = 0

print(dat_dict['type'].value_counts())
dat_dict.reset_index(drop= True)
dat_dict

type
numeric        36
target         10
categorical     4
other           2
Name: count, dtype: int64


,feature,type,modeling_feature,encoded
0,player_id,numeric,0,0
1,player_name,categorical,0,0
2,offsides,numeric,1,0
3,games_minutes,numeric,0,0
4,games_number,numeric,0,0
5,games_position,categorical,0,0
6,games_rating,categorical,0,0
7,games_captain,other,0,0
8,games_substitute,other,0,0
9,shots_total,numeric,1,0


In [11]:
## Encode Features
dat_dict = create_data_index(complete_data,dat_dict,'target',encoding_path)
dat_dict

,feature,type,modeling_feature,encoded
0,player_id,numeric,0,0
1,player_name,categorical,0,0
2,offsides,numeric,1,0
3,games_minutes,numeric,0,0
4,games_number,numeric,0,0
5,games_position,categorical,0,0
6,games_rating,categorical,0,0
7,games_captain,other,0,0
8,games_substitute,other,0,0
9,shots_total,numeric,1,0


In [14]:
['games_position','player_name'] + config["ATTACK_COLS"]

['games_position',
 'player_name',
 'offsides',
 'shots_total',
 'shots_on',
 'goals_total',
 'goals_assists',
 'duels_total',
 'duels_won',
 'dribbles_attempts',
 'dribbles_success',
 'fouls_drawn',
 'penalty_won',
 'dribble_success_rate',
 'target_shot_conversion_perc',
 'duels_won_perc']

In [27]:
# quick knn clustering
from sklearn.cluster import KMeans

# who is ekitike most similar to 

interest_col_set = list(set(['games_position','player_name'] + config["ATTACK_COLS"] +  config["PASSING_COLS"]))

striker_stats = complete_data[interest_col_set].fillna(0)[complete_data[interest_col_set].fillna(0).games_position.isin(['F'])].groupby(['player_name']).agg(n_games = ('player_name','size'),
                                                                          avg_dribble_success = ('dribble_success_rate','mean'),
                                                                          std_dribble_success = ('dribble_success_rate','std'),
                                                                          avg_target_shot_cr = ('target_shot_conversion_perc','mean'),
                                                                          std_target_shot_cr = ('target_shot_conversion_perc','std'),
                                                                          avg_duels_cr = ('duels_won_perc','mean'),
                                                                          std_duels_cr = ('duels_won_perc','std'),
                                                                          avg_pass_accuracy = ('pass_accuracy_perc','mean'),
                                                                          std_pass_accuracy = ('pass_accuracy_perc','std')
                                                                          )


In [28]:
striker_stats = striker_stats[striker_stats.n_games > 30]
striker_stats.shape

(208, 9)

In [29]:
striker_stats

,n_games,avg_dribble_success,std_dribble_success,avg_target_shot_cr,std_target_shot_cr,avg_duels_cr,std_duels_cr,avg_pass_accuracy,std_pass_accuracy
player_name,,,,,,,,,
Abdón Prats,31,1.612903,8.980265,3.225806,17.960530,25.821813,30.392982,45.604986,38.621118
Adam Hložek,50,21.238095,37.777587,8.666667,27.610836,30.744035,32.076038,56.441442,36.109942
Ademola Lookman,33,33.744589,33.843946,28.030303,41.552875,36.661046,22.787809,73.231994,20.763246
Alan Kardec,34,0.000000,0.000000,0.000000,0.000000,23.856209,35.215305,27.225811,40.972115
Alejandro Garnacho,37,24.549550,30.803661,10.360360,28.693611,35.975118,23.977506,78.690810,21.978426
...,...,...,...,...,...,...,...,...,...
William Osula,38,6.140351,23.061822,2.631579,16.222142,12.719298,26.404741,13.157895,30.303246
Yann Karamoh,31,13.010753,28.144532,0.000000,0.000000,22.154727,33.335151,56.923626,36.374167
Yoane Wissa,63,29.047619,41.514469,29.682540,40.417686,47.360964,19.197802,75.732783,12.807012


In [30]:
striker_stats.sort_values('avg_dribble_success',ascending=False)

,n_games,avg_dribble_success,std_dribble_success,avg_target_shot_cr,std_target_shot_cr,avg_duels_cr,std_duels_cr,avg_pass_accuracy,std_pass_accuracy
player_name,,,,,,,,,
Lionel Messi,31,58.882488,23.355850,29.946237,37.343461,49.335136,16.274889,82.696262,4.495736
Jérémy Doku,39,50.177045,39.888581,2.564103,16.012815,46.604274,29.690045,71.237387,32.476969
Florian Wirtz,49,50.046603,23.052184,23.469388,38.477742,42.927937,13.997218,83.161487,6.186007
Eberechi Eze,31,49.074501,30.363596,17.204301,37.387825,50.723769,17.551683,81.347052,7.759145
Charles De Ketelaere,35,47.333333,35.693869,11.904762,30.670990,42.252474,21.359872,78.498351,6.710599
...,...,...,...,...,...,...,...,...,...
Kevin Behrens,32,0.000000,0.000000,3.125000,17.677670,20.000000,33.290295,34.667659,39.367022
José Manuel Arnáiz,32,0.000000,0.000000,0.000000,0.000000,13.764881,31.527415,31.770833,44.320714
Alphadjo Cissè,35,0.000000,0.000000,0.000000,0.000000,1.428571,8.451543,3.809524,16.212294


# Clustering

In [45]:
# Fit all players within 20 buckets
n_clusters = 20


km = KMeans(n_clusters,random_state= 5)
striker_stats['cluster_dribble'] = km.fit_predict(striker_stats[['avg_dribble_success']])

km = KMeans(n_clusters,random_state= 5)
striker_stats['cluster_shot_cr'] = km.fit_predict(striker_stats[['avg_target_shot_cr']])

km = KMeans(n_clusters,random_state= 5)
striker_stats['cluster_duels'] = km.fit_predict(striker_stats[['avg_duels_cr']])

km = KMeans(n_clusters,random_state= 5)
striker_stats['cluster_pass'] = km.fit_predict(striker_stats[['avg_pass_accuracy']])

In [46]:
# check by clusters:
striker_stats.sort_values(["avg_target_shot_cr","cluster_shot_cr"],ascending= [0,0])[["avg_target_shot_cr","std_target_shot_cr","cluster_shot_cr"]].head(15)

,avg_target_shot_cr,std_target_shot_cr,cluster_shot_cr
player_name,,,
Mateo Retegui,42.592593,47.383013,10
Harry Kane,41.182796,42.511858,10
Ante Budimir,39.473684,46.888317,14
Alexander Isak,37.512821,42.151474,14
Romelu Lukaku,36.111111,47.224556,5
Robert Lewandowski,35.727700,42.194497,5
Alexander Sørloth,35.300926,44.460183,5
Chris Wood,34.701493,44.890369,5
Luuk de Jong,34.343434,45.821853,5


In [47]:
striker_stats.sort_values(["avg_dribble_success","cluster_dribble"],ascending= [0,0])[["avg_dribble_success","std_dribble_success","cluster_dribble"]].head(15)

,avg_dribble_success,std_dribble_success,cluster_dribble
player_name,,,
Lionel Messi,58.882488,23.355850,9
Jérémy Doku,50.177045,39.888581,13
Florian Wirtz,50.046603,23.052184,13
Eberechi Eze,49.074501,30.363596,13
Charles De Ketelaere,47.333333,35.693869,2
Lamine Yamal,45.591956,35.159846,2
Ousmane Dembélé,43.978938,34.511310,15
Luis Díaz,43.840246,37.042307,15
Bukayo Saka,43.764791,34.560404,15


In [48]:
striker_stats.sort_values(["avg_duels_cr","cluster_duels"],ascending= [0,0])[["avg_duels_cr","std_duels_cr","cluster_duels"]].head(15)

,avg_duels_cr,std_duels_cr,cluster_duels
player_name,,,
Tobias Lauritsen,59.122808,12.757295,16
Lamine Yamal,56.314001,20.889237,9
Thierno Barry,54.228578,18.863703,9
Luuk de Jong,53.802408,21.541981,9
Chuba Akpom,53.395367,23.921561,9
Eberechi Eze,50.723769,17.551683,1
Anthony Gordon,50.394848,19.133019,1
Benjamin Šeško,49.744306,20.755658,1
Juanmi Latasa,49.716603,27.856468,1


In [49]:
striker_stats.sort_values(["avg_pass_accuracy","cluster_pass"],ascending= [0,0])[["avg_pass_accuracy","std_pass_accuracy","cluster_pass"]].head(15)

,avg_pass_accuracy,std_pass_accuracy,cluster_pass
player_name,,,
Gianluca Caprari,86.384669,17.201659,7
Jonathan David,83.532435,9.704329,7
Florian Wirtz,83.161487,6.186007,7
Rodrygo,82.934651,23.803298,7
Lionel Messi,82.696262,4.495736,7
Bukayo Saka,82.469733,8.108529,7
Amine Gouiri,82.347721,11.388350,7
Kylian Mbappé,82.319710,13.696919,7
Anthony Gordon,81.521477,9.556990,7


In [54]:
complete_data.head()

,player_id,player_name,offsides,games_minutes,games_number,games_position,games_rating,games_captain,games_substitute,shots_total,shots_on,goals_total,goals_conceded,goals_assists,goals_saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved,fixture_id,team_goals_scored,team_non_penalty_goals_scored,team_goals_scored_half,team_goals_conceded,team_non_penalty_goals_conceded,team_goals_conceded_half,opponent,team_winner,team,outcome,dribble_success_rate,target_shot_conversion_perc,duels_won_perc,pass_accuracy_perc,outcome_num,win
0,47269,Álex Remiro,NaN,90.0,1,G,6.2,False,False,NaN,NaN,NaN,2,0.0,2.0,31.0,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,0.0,1208500,1,1.0,0,2,2.0,0,Rayo Vallecano,Rayo Vallecano,Real Sociedad,loss,NaN,NaN,NaN,83.870968,1,0
1,2202,Hamari Traoré,NaN,75.0,18,D,6.9,False,False,NaN,NaN,NaN,0,0.0,NaN,37.0,NaN,30.0,2.0,NaN,NaN,8.0,5.0,1.0,1.0,1.0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,1208500,1,1.0,0,2,2.0,0,Rayo Vallecano,Rayo Vallecano,Real Sociedad,loss,100.0,NaN,62.500000,81.081081,1,0
2,47298,Aritz Elustondo,NaN,90.0,6,D,6.9,False,False,1.0,NaN,NaN,0,0.0,NaN,77.0,1.0,70.0,1.0,1.0,1.0,10.0,6.0,NaN,NaN,NaN,4.0,2.0,0,0,NaN,NaN,0,0,NaN,1208500,1,1.0,0,2,2.0,0,Rayo Vallecano,Rayo Vallecano,Real Sociedad,loss,NaN,NaN,60.000000,90.909091,1,0
3,182546,Jon Pacheco,NaN,90.0,20,D,7.5,False,False,NaN,NaN,NaN,0,0.0,NaN,51.0,NaN,45.0,1.0,1.0,1.0,6.0,4.0,NaN,NaN,NaN,1.0,1.0,0,0,NaN,NaN,0,0,NaN,1208500,1,1.0,0,2,2.0,0,Rayo Vallecano,Rayo Vallecano,Real Sociedad,loss,NaN,NaN,66.666667,88.235294,1,0
4,182636,Javi López,NaN,90.0,12,D,5.9,False,False,1.0,NaN,NaN,0,0.0,NaN,39.0,NaN,34.0,NaN,1.0,NaN,5.0,1.0,1.0,NaN,1.0,1.0,NaN,0,0,NaN,NaN,0,0,NaN,1208500,1,1.0,0,2,2.0,0,Rayo Vallecano,Rayo Vallecano,Real Sociedad,loss,NaN,NaN,20.000000,87.179487,1,0


In [51]:
complete_data[complete_data.player_name.str.contains("Darwin")]['player_name'].unique()

array(['Darwin Machís', 'Darwin Núñez'], dtype=object)

In [52]:
player_to_check = "Anthony Gordon"
striker_stats[striker_stats.index.isin(["Anthony Gordon","Alexander Isak","Hugo Ekitiké","Darwin Núñez"]) ]

,n_games,avg_dribble_success,std_dribble_success,avg_target_shot_cr,std_target_shot_cr,avg_duels_cr,std_duels_cr,avg_pass_accuracy,std_pass_accuracy,cluster_dribble,cluster_shot_cr,cluster_duels,cluster_pass
player_name,,,,,,,,,,,,,
Alexander Isak,65,41.008547,37.573602,37.512821,42.151474,33.051244,20.340532,77.344333,12.969312,15,14,0,14
Anthony Gordon,67,37.777778,35.678417,20.398010,37.808317,50.394848,19.133019,81.521477,9.556990,11,11,1,7
Darwin Núñez,72,19.444444,36.364526,12.268519,28.662552,33.162288,24.059192,62.001571,27.581762,6,3,0,15
Hugo Ekitiké,73,29.230267,35.062853,15.753425,32.854244,28.859292,25.762574,62.143706,34.113981,7,19,5,15


In [ ]:
filter_query = 'games_position.isin(["F"])'


# Stat to look at:
stat = 'target_shot_conversion_perc'

# configs 
min_appearance = 40

dribble_dat_g = complete_data[complete_data.games_position.isin(['F'])].reset_index().groupby("player_name").agg(n_apps = ("player_name","size"),stat = (stat,"median")).reset_index()
dribble_dat_g = dribble_dat_g[dribble_dat_g.n_apps >= min_appearance]
dribble_dat_g['rank'] = dribble_dat_g["stat"].fillna(0).rank(ascending= False,method = 'dense')
dribble_dat_g.sort_values("rank",inplace = True)

fig, ax = plt.subplots(figsize=(13, 8))

# Plot correctly, no comma here
sns.boxplot(
    data=dribble_dat[dribble_dat.player_name.isin(dribble_dat_g[dribble_dat_g['rank'] < 15]['player_name'])],
    x="player_name",
    y=stat,
    order=dribble_dat_g[dribble_dat_g['rank'] < 15]['player_name'],
    ax=ax,
    
)

# Now this works correctly on `ax`
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_title(f"Stat: {stat}")
plt.tight_layout()
plt.show()


In [ ]:
complete_data[['outcome_num','outcome']].value_counts()

In [ ]:
fig = plt.subplots(nrows=1, ncols = 1, figsize = (20,10))
fig = sns.heatmap(complete_data.query(filter_query)[config['ATTACK_COLS'] + ['team_goals_scored','team_non_penalty_goals_scored','team_goals_conceded']].corr(),cmap = 'coolwarm')
fig.set_xticklabels(fig.get_xticklabels(),rotation = 60)


In [ ]:
# trial multiclass model:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(complete_data[list(set(config['DEFENSE_COLS'] + config['PASSING_COLS'] + config['ATTACK_COLS'])) + ['outcome_num']].drop(columns = 'outcome_num'),complete_data[list(set(config['DEFENSE_COLS'] + config['PASSING_COLS'] + config['ATTACK_COLS'])) + ['outcome_num']][['outcome_num']],stratify=nn_dat[['outcome_num']],random_state=33)


In [ ]:
dat = NNDataFromPd(X_train.fillna(0), y_train.outcome_num, dat_dict)
train_loader = DataLoader(dat, batch_size = 128,shuffle= True)

In [ ]:
train_loader.dataset.X_numeric_tensor.shape

In [ ]:
# model params
n_features = X_train.shape[1]
n_classes = y_train.iloc[:,0].nunique()
model = MultiClassModel(n_features,n_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
epochs = 500

for epoch in range(epochs):
    
    epoch_loss = 0

    for X_numeric_batch, X_categoric_batch, y_batch in train_loader:
        
        pred = model.forward(X_numeric_batch)
        
        loss = criterion(pred,y_batch)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch: {epoch}, Loss: {epoch_loss}")

In [ ]:
model.eval()

In [ ]:
X_test['passes_accuracy'] = X_test['passes_accuracy'].astype("float64")

In [ ]:
# test sets

test_dat = NNDataFromPd(X_test,y_test,dat_dict)
test_loader = DataLoader(test_dat,batch_size= X_test.shape[0],shuffle=True)

In [ ]:
model.eval()
with torch.no_grad():
    for X_numeric_batch, X_categoric_batch, y_batch in test_loader:
        output = model(X_numeric_batch)
        pred_class = torch.argmax(output, dim = 1)

In [ ]:
# Logistic Model:
X_train, X_test, y_train, y_test = train_test_split(complete_data[complete_data.games_position == 'F'][list(set(config['DEFENSE_COLS'] + config['PASSING_COLS'] + config['ATTACK_COLS'])) + ['win']].drop(columns = 'win'),
                                                    complete_data[complete_data.games_position == 'F']['win'],
                                                    stratify=complete_data['win'],
                                                    random_state=33)

In [ ]:
train_dat = NNDataFromPd(X_train,y_train,dat_dict)
train_loader = DataLoader(train_dat,batch_size= 128,shuffle = True)

In [ ]:
n_features = X_train.shape[1]
model = LogisticNNModelComplex(n_features)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.005)

In [ ]:
epochs = 500
for epoch in range(epochs):
    epoch_loss = 0
    
    for X_numeric, X_categoric, y in train_loader:

        pred = model(X_numeric)

        loss = criterion(pred,y.unsqueeze(1))

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch},  Loss: {loss}")


In [ ]:
pred_proba.squeeze(1)

In [ ]:
from validations import *


test_dat = NNDataFromPd(X_test,y_test,dat_dict)
test_loader = DataLoader(test_dat,batch_size= X_test.shape[0],shuffle=True)

model.eval()
with torch.no_grad():
    for X_numeric_batch, X_categoric_batch, y_batch in test_loader:
        output = model(X_numeric_batch)
        pred_proba = torch.softmax(output,dim =1)
        pred_class = torch.argmax(output, dim = 1)


discrete_evaluations(y_test,pred_class,pred_proba.squeeze(1),classification_type="Binary",model_path= output_path)